In [ ]:
import json
from pathlib import Path
import sys



sys.path.append(str(Path.cwd().parent))

from libraries import browser
from libraries.client_stashapp import StashAppClient, get_stashapp_client
from libraries.stash_check_builder import create_check, fix, performer_tags, query, studios, tags


# Browser functionality for injecting fix data
def check_browser_connection():
    """Check if Chrome is running with remote debugging enabled"""
    try:
        import requests
        response = requests.get("http://localhost:9222/json", timeout=5)
        if response.status_code == 200:
            pages = json.loads(response.text)
            print(f"✅ Chrome is running with remote debugging")
            print(f"📊 Found {len(pages)} open tabs/pages")
            return True
        else:
            print(f"❌ Chrome debugging port responded with status {response.status_code}")
            return False
    except Exception as e:
        print("❌ Cannot connect to Chrome debugging port")
        print("🔧 Please start Chrome with: --remote-debugging-port=9222")
        return False

def create_scenes_query_url(url_query):
    """Create a Stash query URL for scenes with Facial tag but excluding Cumshot tag"""
    import urllib.parse

    url = f"{base_url}/scenes?sortby=path&perPage=60"

    for component in url_query:
        # Convert to JSON string and use the specific URL encoding format that Stash expects
        query_json = json.dumps(component)
        # Replace curly braces with parentheses and square brackets with %5B %5D
        query_str = query_json.replace("{", "(").replace("}", ")").replace("[", "%5B").replace("]", "%5D")
        # URL encode the query string
        query_str = "&c=" + urllib.parse.quote(query_str, safe="():%5B%5D")
        url += query_str

    return url

def open_scenes_with_fix_data(check, check_matching_scenes):
    """Open the problematic scenes in browser with fix data injected"""

    if not check_matching_scenes:
        print("⚠️  No scenes found that need fixing")
        return False

    # Create URL for the scenes
    url = create_scenes_query_url(check["url_query"])

    print(f"🚀 Opening browser with {len(check_matching_scenes)} scenes that need to be fixed...")
    print(f"📋 Fix data will be injected as window.stashFixesData")
    print(f"🔗 URL: {url}")

    # Open the browser tab with fix data
    results = browser.open_or_update_tabs_with_data([url], check["fixes"], "stashFixesData")

    success = results.get(url, False)
    if success:
        print("✅ Successfully opened browser tab with fix data!")
    else:
        print("❌ Failed to open browser tab")

    return success

stash = get_stashapp_client()
stash_client = StashAppClient()

base_url = "https://stash.chiefsclub.com"
checks = []

In [ ]:
name = "Facial without Cumshot"
checks.append(
    create_check()
    .name(name)
    .query(
        query().tags(
            tags()
            .include("Facial")
            .exclude("Cumshot", "Fake Cum")
            .modifier("INCLUDES_ALL")
        )
    )
    .fix(
        fix("Missing Cumshot Fix")
        .add_tags("Cumshot")
    )
    .build(stash)
)

print(f"✅ Created check: {name}")

In [ ]:
name = "Both Straight and Lesbian without Multiple Sex Scenes In A Scene"
checks.append(
    create_check()
    .name(name)
    .query(
        query().tags(
            tags()
            .include("Twosome (Straight)", "Twosome (Lesbian)")
            .exclude("Multiple Sex Scenes In A Scene")
            .modifier("INCLUDES_ALL")
            .depth(0)
        )
    )
    .fix(
        fix("Remove Twosome (Straight)")
        .remove_tags("Twosome (Straight)")
    )
    .fix(
        fix("Remove Twosome (Lesbian)")
        .remove_tags("Twosome (Lesbian)")
    )
    .build(stash)
)

print(f"✅ Created check: {name}")

In [ ]:
name = "Both BBG and BGG without Multiple Sex Scenes In A Scene"
checks.append(
    create_check()
    .name(name)
    .query(
        query().tags(
            tags()
            .include("Threesome (BBG)", "Threesome (BGG)")
            .exclude("Multiple Sex Scenes In A Scene")
            .modifier("INCLUDES_ALL")
            .depth(0)
        )
    )
    .fix(
        fix("Remove Threesome (BBG)")
        .remove_tags("Threesome (BBG)")
    )
    .fix(
        fix("Remove Threesome (BGG)")
        .remove_tags("Threesome (BGG)")
    )
    .build(stash)
)

print(f"✅ Created check: {name}")

In [ ]:
name = "Data Quality: 2 Performers No Group Makeup, Tags Indicate Straight"
checks.append(
    create_check()
    .name(name)
    .query(
        query()
        .performer_count("EQUALS", 2)
        .tags(
            tags()
            .modifier("INCLUDES")
            .include("Blowjob", "Facial", "Cumshot")
            .exclude("Behind the Scenes", "Full Movie", "Category: Group Makeup", "Futanari")
            .depth(-1)
        )
    )
    .fix(
        fix("Add Group Makeup Verified, Twosome, Twosome (Straight)")
        .add_tags("Group Makeup Verified", "Twosome", "Twosome (Straight)")
    )
    .build(stash)
)

print(f"✅ Created check: {name}")

In [ ]:
name = "Data Quality: Straight scene with Transgender Female performer"
checks.append(
    create_check()
    .name(name)
    .query(
        query()
        .performer_tags(
            performer_tags()
            .include("Transgender Female")
            .modifier("INCLUDES")
            .depth(0)
        )
        .tags(
            tags()
            .include("Straight", "Twosome (Straight)")
            .exclude("Multiple Sex Scenes In A Scene")
            .modifier("INCLUDES_ALL")
            .depth(0)
        )
    )
    .fix(
        fix("Remove Straight, Twosome (Straight)")
        .remove_tags("Straight", "Twosome (Straight)")
    )
    .build(stash)
)

print(f"✅ Created check: {name}")

In [ ]:
name = "Data Quality: Futanari XXX without Futanari tag"
checks.append(
    create_check()
    .name(name)
    .query(
        query()
        .tags(
            tags()
            .exclude("Futanari")
            .modifier("INCLUDES")
            .depth(0)
        )
        .studios(
            studios()
            .include("Futanari XXX")
            .modifier("INCLUDES")
            .depth(0)
        )
    )
    .fix(
        fix("Add Futanari tag")
        .add_tags("Futanari")
    )
    .build(stash)
)

print(f"✅ Created check: {name}")

In [ ]:
name = "Lesbian tag in Straight scene"
checks.append(
    create_check()
    .name(name)
    .query(
        query().tags(
            tags()
            .include("Lesbian", "Twosome (Straight)")
            .exclude("Straight", "Multiple Sex Scenes In A Scene", "Behind the Scenes", "Full Movie")
            .modifier("INCLUDES_ALL")
            .depth(0)
        )
    )
    .fix(
        fix("Remove Lesbian tag")
        .remove_tags("Lesbian")
    )
    .build(stash)
)

print(f"✅ Created check: {name}")

In [ ]:
name = "Unverified group makeup for scene with Solo, Solo Female, Group Makeup Calculated: 1F"
checks.append(
    create_check()
    .name(name)
    .query(
        query()
        .performer_count("EQUALS", 1)
        .performer_tags(
            performer_tags()
            .include("Female")
            .modifier("INCLUDES")
            .depth(0)
        )
        .tags(
            tags()
            .include("Solo", "Solo Female", "Group Makeup Calculated: 1F")
            .exclude("Group Makeup Verified")
            .modifier("INCLUDES_ALL")
            .depth(0)
        )
    )
    .fix(
        fix("Add Group Makeup Verified tag")
        .add_tags("Group Makeup Verified")
        .remove_tags("Group Makeup Calculated: 1F")
    )
    .build(stash)
)

print(f"✅ Created check: {name}")

In [ ]:
name = "Unverified group makeup for scene with Twosome, Twosome (Straight), Group Makeup Calculated: 1F1M"
checks.append(
    create_check()
    .name(name)
    .query(
        query()
        .performer_count("EQUALS", 2)
        .performer_tags(
            performer_tags()
            .include("Female", "Male")
            .modifier("INCLUDES")
            .depth(0)
        )
        .tags(
            tags()
            .include("Twosome", "Twosome (Straight)", "Group Makeup Calculated: 1F1M")
            .exclude("Group Makeup Verified")
            .modifier("INCLUDES_ALL")
            .depth(0)
        )
    )
    .fix(
        fix("Add Group Makeup Verified tag")
        .add_tags("Group Makeup Verified")
        .remove_tags("Group Makeup Calculated: 1F1M")
    )
    .build(stash)
)

print(f"✅ Created check: {name}")

In [ ]:
stash_tags = stash_client.get_tags_by_names(
    [
        "Group Makeup Generic",
        "Group Makeup Specific",
    ]
)

generic_group_makeup_tags = stash.find_tags(
    {
        "parents": {
            "value": [stash_tags.group_makeup_generic["id"]],
            "modifier": "INCLUDES",
            "depth": -1,
        }
    },
    fragment="id name",
)
specific_group_makeup_tags = stash.find_tags(
    {
        "parents": {
            "value": [stash_tags.group_makeup_specific["id"]],
            "modifier": "INCLUDES",
            "depth": -1,
        }
    },
    fragment="id name",
)

tags_to_be_removed = (
    [tag["name"] for tag in generic_group_makeup_tags]
    + [tag["name"] for tag in specific_group_makeup_tags]
    + [
        "Group Makeup Verified",
        "Data Quality: Multiple Generic Group Makeup Tags",
        "Data Quality: Multiple Specific Group Makeup Tags",
    ]
)

name = "Multiple generic group makeup tags"
checks.append(
    create_check()
    .name(name)
    .query(
        query().tags(
            tags()
            .include("Data Quality: Multiple Generic Group Makeup Tags", "Data Quality: Multiple Specific Group Makeup Tags")
            .modifier("INCLUDES")
            .depth(0)
        )
    )
    .fix(
        fix("Remove all group makeup tags")
        .remove_tags(*tags_to_be_removed)
    )
    .build(stash)
)

print(f"✅ Created check: {name}")

In [ ]:
name = "Group Makeup Verified without Group Makeup Generic or Specific"
checks.append(
    create_check()
    .name(name)
    .query(
        query().tags(
            tags()
            .include("Group Makeup Verified")
            .exclude("Group Makeup Generic", "Group Makeup Specific")
            .modifier("INCLUDES")
            .depth(-1)
        )
    )
    .fix(
        fix("Remove Group Makeup Verified tag")
        .remove_tags("Group Makeup Verified")
    )
    .build(stash)
)

print(f"✅ Created check: {name}")

In [ ]:
name = "Missing Performer (Male) but without Male Difficult To Identify"
checks.append(
    create_check()
    .name(name)
    .query(
        query().tags(
            tags()
            .include("Missing Performer (Male)")
            .exclude("Male Difficult To Identify")
            .modifier("INCLUDES")
            .depth(-1)
        )
    )
    .fix(
        fix("Add Male Difficult To Identify tag")
        .add_tags("Male Difficult To Identify")
    )
    .build(stash)
)

print(f"✅ Created check: {name}")

In [ ]:
checks_with_matching_scenes = []

for check in checks:
    matching_scenes = stash.find_scenes(check["query"], fragment=check["fragment"])
    print(f"Found {len(matching_scenes)} scenes for {check['name']}")
    if len(matching_scenes) > 0:
        checks_with_matching_scenes.append(check)

In [ ]:
# Check browser connection
browser_ready = check_browser_connection()

current_check = checks_with_matching_scenes[-1]
current_matching_scenes = stash.find_scenes(
    current_check["query"], fragment=current_check["fragment"]
)

if not browser_ready:
    print("Warning: Browser not ready. Start Chrome with:")
    print(
        'Start-Process "chrome.exe" -ArgumentList'
        ' "--remote-debugging-port=9222",'
        ' "--user-data-dir=C:\\\\temp\\\\chrome-debug",'
        ' "--remote-allow-origins=*"'
    )
else:
    open_scenes_with_fix_data(current_check, current_matching_scenes)